In [2]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [3]:
# Import the data
batchsize = 8
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)


In [5]:

train_data = train_datagen.flow_from_directory(r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\train', target_size=(80,80),
                                               batch_size=batchsize, class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\train', target_size=(80,80),
                                               batch_size=batchsize, class_mode='categorical', subset='validation')

Found 64105 images belonging to 2 classes.
Found 16025 images belonging to 2 classes.


In [7]:
test_datagen= ImageDataGenerator(rescale= 1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\test', target_size=(80,80),
                                               batch_size=batchsize, class_mode='categorical')

Found 4768 images belonging to 2 classes.


In [8]:
# Model training
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)

for layer in bmodel.layers:
    layer.trainable = False

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 80, 80, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 39, 39, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 39, 39, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 39, 39, 32)           0         ['batch_normalization[0][0

In [10]:
checkpoint = ModelCheckpoint(
    r'C:\Users\INFINIX\Driver Drowsiness Detection System\models\model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=3
)

earlystop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    verbose=3,
    restore_best_weights=True
)

learning_rate = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=3,
)

callbacks = [checkpoint, earlystop, learning_rate]

In [11]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(
    train_data,
    steps_per_epoch=train_data.samples//batchsize,
    validation_data=validation_data,
    validation_steps=validation_data.samples//batchsize,
    callbacks=callbacks,
    epochs=5
)


C:\Users\INFINIX\AppData\Local\Temp\ipykernel_3516\2642282653.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/5


8013/8013 [==============================] - ETA: 0s - loss: 0.2091 - accuracy: 0.9179
Epoch 1: val_loss improved from inf to 0.16732, saving model to C:\Users\INFINIX\Driver Drowsiness Detection System\models\model.h5


C:\Users\INFINIX\.conda\envs\yogesh\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8013/8013 [==============================] - 1110s 137ms/step - loss: 0.2091 - accuracy: 0.9179 - val_loss: 0.1673 - val_accuracy: 0.9305 - lr: 0.0010
Epoch 2/5
 369/8013 [>.............................] - ETA: 12:27 - loss: 0.1859 - accuracy: 0.9231

KeyboardInterrupt: 

In [12]:
# Extract features for SVM
train_features = model.predict(train_data)
train_labels = train_data.classes

test_features = model.predict(test_data)
test_labels = test_data.classes


596/596 [==============================] - 54s 90ms/step


In [14]:
# Train SVM
svm_model = SVC()
svm_model.fit(train_features, train_labels)
svm_predictions = svm_model.predict(test_features)
svm_accuracy = accuracy_score(test_labels, svm_predictions)


In [15]:
print("SVM Accuracy:", svm_accuracy)

SVM Accuracy: 0.516988255033557


In [ ]:
# Evaluate the model
loss_tr, acc_tr = model.evaluate(train_data)
print("Training Accuracy:", acc_tr)
print("Training Loss:", loss_tr)

In [ ]:
acc_test, loss_test = model.evaluate(test_data)
print("Test Accuracy:", acc_test)
print("Test Loss:", loss_test)